In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('data/clean/train_clean_v2.csv')
df_targets = pd.read_csv('data/clean/train_labels.csv')

In [ ]:
df.shape, df_targets.shape

In [4]:
df_copy = df.copy()

In [ ]:
#cols of the preview months
new_cols = [s+'_prev_month' for s in df_targets.columns.tolist()]

In [5]:
groupby_fecha_dato = df_copy.groupby(['fecha_dato'])

In [6]:
dates = df_copy.fecha_dato.unique()
dates

array(['2015-01-28', '2015-02-28', '2015-03-28', '2015-04-28',
       '2015-05-28', '2015-06-28', '2015-07-28', '2015-08-28',
       '2015-09-28', '2015-10-28', '2015-11-28', '2015-12-28',
       '2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28'], dtype=object)

### Prueba individual
Tomar los productos del mes anterior

In [90]:
p = groupby_fecha_dato.get_group(dates[0]) #mes anterior
ps = groupby_fecha_dato.get_group(dates[1]) #mes actual

In [91]:
p.shape, ps.shape

((513791, 27), (516199, 27))

#### Para clientes que están en el mes actual y en el anterior
En esta parte hay clientes que ya no están en el mes actual

In [105]:
list(set(p.ncodpers.values) - set(ps.ncodpers.values))[1]

206851

In [ ]:
codes_in_p_and_ps = list(set(ps.ncodpers.values) & set(p.ncodpers.values)) #clientes que están en ambos meses
len(codes_in_p_and_ps)

In [ ]:
df_aux = df_targets.loc[p.ncodpers.loc[p.ncodpers.isin(codes_in_p_and_ps)].index]
df_aux = df_aux.join(df_copy.ncodpers, how='inner')
df_aux = df_aux[[df_aux.columns.tolist()[-1]] + df_aux.columns.tolist()[:-1]]

In [ ]:
ps = ps.merge(df_aux, on='ncodpers', how='outer', right_index=True)

---
### Nuevo dataset
En esta dataset se agregan como *features* los productos que el cliente tenía en el mes anterior, además hay que tener en cuenta lo siguiente: los clientes que son "nuevos" o no aparecen en el mes anterior se llena de ceros

In [70]:
%%time
df_new = pd.DataFrame(columns=df_copy.columns.tolist() + df_targets.columns.tolist())
for i in range(1, len(dates)):
    pre = groupby_fecha_dato.get_group(dates[i-1]) #mes anterior
    pos = groupby_fecha_dato.get_group(dates[i]) #mes actual
    
    codes_in_pre_and_pos = list(set(pos.ncodpers.values) & set(pre.ncodpers.values)) #clientes que están en ambos meses
    
    df_aux = df_targets.loc[pre.ncodpers.loc[pre.ncodpers.isin(codes_in_pre_and_pos)].index]
    df_aux = df_aux.join(df_copy.ncodpers, how='inner')
    df_aux = df_aux[[df_aux.columns.tolist()[-1]] + df_aux.columns.tolist()[:-1]]
    
    pos = pos.merge(df_aux, on='ncodpers', how='outer', right_index=True)
    pos.fillna(0, inplace=True)
    df_new = df_new.append(pos)

CPU times: user 1min 43s, sys: 1min 27s, total: 3min 11s
Wall time: 3min 11s


In [118]:
df_targets.loc[df_new.index].shape

(10281601, 24)

In [113]:
x_train = df_new.drop(['fecha_dato', 'fecha_alta', 'ncodpers'], axis=1).as_matrix()

In [115]:
x_train.shape

(10281601, 48)

In [119]:
df_targets.shape

(10795392, 24)

In [122]:
y_df = df_targets.loc[df_new.index]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.])